**ЛИНЕЙНАЯ РЕГРЕССИЯ**

1. Даны значения величины заработной платы заемщиков банка (zp) и значения их поведенческого кредитного скоринга (ks): zp = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110], ks = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]. Используя математические операции, посчитать коэффициенты линейной регрессии, приняв за X заработную плату (то есть, zp - признак), а за y - значения скорингового балла (то есть, ks - целевая переменная). Произвести расчет как с использованием intercept, так и без.

In [20]:
import threading

In [4]:
import numpy as np
import scipy.stats as stats

In [5]:
%precision 4

'%.4f'

In [20]:
zp = np.array([35, 45, 190, 200, 40, 70, 54, 150, 120, 110]) # это Х - независимая переменная
ks = np.array([401, 574, 874, 919, 459, 739, 653, 902, 746, 832]) # целевая переменная

$\hat y = \beta_0+\beta_1*X$ &emsp; -> с интерсептом &emsp;&emsp; $\hat y = \beta_1*X$&emsp;-> без интерсепта--> 
$\begin{pmatrix}y_1\\y_2\\ \dots \\y_n \end{pmatrix} = \begin{pmatrix}x_1\\x_2\\ \dots \\x_n \end{pmatrix}\begin{pmatrix}  \\ \beta_1 \\   \\ \end{pmatrix}$

In [87]:
b = (np.mean(zp*ks) - np.mean(zp)*np.mean(ks))/(np.mean(zp**2) - np.mean(zp)**2)
b

2.6205

In [88]:
a = np.mean(ks)-b*np.mean(zp)
a

444.1774

In [16]:
X = zp.reshape(10, 1)
Y = ks.reshape(10, 1)
X, Y

(array([[ 35],
        [ 45],
        [190],
        [200],
        [ 40],
        [ 70],
        [ 54],
        [150],
        [120],
        [110]]),
 array([[401],
        [574],
        [874],
        [919],
        [459],
        [739],
        [653],
        [902],
        [746],
        [832]]))

$ \hat B = (X^T*X)^{-1}* X^T*Y$

$\begin{pmatrix}y_1\\y_2\\ \dots \\y_n \end{pmatrix} = \begin{pmatrix}1 & x_1\\1 & x_2\\ \dots & \dots \\1 & x_n \end{pmatrix}\begin{pmatrix} \beta_0 \\ \beta_1 \\ \end{pmatrix}$ &emsp; --> с интерсептом

In [90]:
B = np.dot(np.linalg.inv(np.dot(X.T, X)), X.T @ ks) # без интерсепта
B

array([5.8898])

In [91]:
X1 = np.hstack([np.ones((10, 1)),X])
X1

array([[  1.,  35.],
       [  1.,  45.],
       [  1., 190.],
       [  1., 200.],
       [  1.,  40.],
       [  1.,  70.],
       [  1.,  54.],
       [  1., 150.],
       [  1., 120.],
       [  1., 110.]])

In [92]:
B1 = np.dot(np.linalg.inv(np.dot(X1.T, X1)), X1.T @ ks)
B1

array([444.1774,   2.6205])

2. Посчитать коэффициент линейной регрессии при заработной плате (zp), используя градиентный спуск (без intercept).

In [13]:
def mse_(B1, ks=ks, zp=zp, n = 10):
    return np.sum((B1*zp-ks)**2)/n

In [7]:
B1 = 0.1
alpha = 2e-5 # если обучение идет медленно, то либо добавляем количесвто итераций, либо увеличиваем alpha
n = len(ks)

In [16]:
%%time
for i in range(300000):
    B1 -= alpha*(2/n)*np.sum((B1*zp-ks)*zp)
    if i%75000 ==0:
        print(f'Iteration: {i}, B1={B1}, mse = {mse_(B1)}');

Iteration: 0, B1=5.889820420132689, mse = 56516.85841571941
Iteration: 75000, B1=5.889820420132689, mse = 56516.85841571941
Iteration: 150000, B1=5.889820420132689, mse = 56516.85841571941
Iteration: 225000, B1=5.889820420132689, mse = 56516.85841571941
CPU times: total: 3.91 s
Wall time: 3.91 s


In [39]:
B1 = 0.1
B0 = 0.1

In [40]:
def mse(B0, B1, ks=ks, zp=zp, n = 10):
    return np.sum((B0+B1*zp-ks)**2)/n

In [56]:
%%time
class CustomProcess(Process):
    z = 4000001
    def main(*args):
        B1 = 0.1
        B0 = 0.1
        for i in range(z):
            y_pred = B0+B1*zp
            B0 -= alpha*(2/n)*np.sum((y_pred-ks))
            B1 -= alpha*(2/n)*np.sum((y_pred-ks)*zp)
            if i%1000000 ==0:
                print(f'Iteration: {i}, B0 = {B0}, B1={B1}, mse = {mse(B0,B1)}')
print(main(z))
if __name__ == '__main__':
    p1 = CustomProcess()
    p1.start()
    p1.join()

Iteration: 0, B0 = 0.1279864, B1=3.2901560000000005, mse = 149526.33499003082
Iteration: 1000000, B0 = 444.1599385302417, B1=2.6206670924287647, mse = 6470.414278142239
Iteration: 2000000, B0 = 444.1773566410366, B1=2.6205388874323243, mse = 6470.414201176665
Iteration: 3000000, B0 = 444.1773573215585, B1=2.6205388824233826, mse = 6470.414201176654
Iteration: 4000000, B0 = 444.1773573215585, B1=2.6205388824233826, mse = 6470.414201176654
None
CPU times: total: 1min 38s
Wall time: 1min 39s


In [19]:
zp = np.hstack([np.ones((10, 1)),zp])
ks = np.hstack([np.ones((10, 1)),ks])
zp

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [11]:
import timeit
import time
import multiprocessing as mp
from contextlib import contextmanager
from sklearn.linear_model import LinearRegression, Ridge, Lasso


def generate_data():
    X = 0.1
    y = 0.1
    for i in range(10):
        y_pred = X+y*zp
        X -= alpha*(2/n)*np.sum((y_pred-ks))
        y -= alpha*(2/n)*np.sum((y_pred-ks)*zp)
    return X, y

@contextmanager
def timeit(tag, container=None):
    t0 = time.perf_counter()
    yield
    t1 = time.perf_counter()
    if container is None:
        print('[{}] : {:.2f} seconds'.format(tag, t1 - t0))
    else:
        container[tag] = t1 - t0

def test(i):
    X, y = generate_data()
    m = LinearRegression()    # Ridge, Lasso produce similar results
    with timeit('OLS.fit {}'.format(i)):
        m.fit(X, y)

# print('===================== MultiProcessing =====================')
# with mp.Pool(8) as pool:
#     pool.map(test, range(10))

print('===================== For Loop =====================')
for i in range(1000):
    test(i)

===================== For Loop =====================


ValueError: Expected 2D array, got scalar array instead:
array=0.18605610648411341.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [1]:
# import threading
# threading.active_count()
from multiprocessing import cpu_count
cpu_count()

8

In [110]:
%whos

Variable   Type        Data/Info
--------------------------------
B          ndarray     1: 1 elems, type `float64`, 8 bytes
B0         float64     444.1773573215585
B1         float64     2.6205388824233826
X          ndarray     10x1: 10 elems, type `int32`, 40 bytes
X1         ndarray     10x2: 20 elems, type `float64`, 160 bytes
Y          ndarray     10x1: 10 elems, type `int32`, 40 bytes
a          float64     444.1773573243596
alpha      float       2e-05
b          float64     2.620538882402765
i          int         4000000
ks         ndarray     10: 10 elems, type `int32`, 40 bytes
mse        function    <function mse at 0x000001DAAC8969D0>
mse_       function    <function mse_ at 0x000001DAAD8344C0>
n          int         10
np         module      <module 'numpy' from 'C:\<...>ges\\numpy\\__init__.py'>
stats      module      <module 'scipy.stats' fro<...>ipy\\stats\\__init__.py'>
y_pred     ndarray     10: 10 elems, type `float64`, 80 bytes
zp         ndarray     10: 10 elem

In [116]:
%pinfo2 mse

Signature:
mse(
    B0,
    B1,
    ks=array([401, 574, 874, 919, 459, 739, 653, 902, 746, 832]),
    zp=array([ 35,  45, 190, 200,  40,  70,  54, 150, 120, 110]),
    n=10,
)
Docstring: <no docstring>
Source:   
def mse(B0, B1, ks=ks, zp=zp, n = 10):
    return np.sum((B0+B1*zp-ks)**2)/n
File:      c:\users\phyzician\appdata\local\temp\ipykernel_13760\3533577948.py
Type:      function


In [52]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
import time

X, y = datasets.make_classification(n_samples=10000, n_features=50, n_informative=20, n_classes=10)

start = time.time()
model = RandomForestClassifier(n_estimators=500, n_jobs=8)
model.fit(X, y)
print(f'Time: {time.time()-start:.2f} sec')

Time: 6.64 sec


In [53]:
from multiprocessing import Process
import os
X, y = datasets.make_classification(n_samples=10000, n_features=50, n_informative=20, n_classes=10)
def main(*args):
    start = time.time()
    model = RandomForestClassifier(n_estimators=500, n_jobs=8)
    model.fit(X, y)
    w = time.time()-start
    return w

print(main(X,y))

if __name__ == '__main__':
    p1 = Process(target=main, args=(1,))
    p2 = Process(target=main, args=(2,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()

6.675304651260376
